In [1]:
import torch
import random
import numpy as np
from functools import partial

In [2]:
# pip install torch torch_geometric torch-scatter pandas scikit-learn wandb dill openpyxl torchmetrics
# python -m pip install gurobipy

In [3]:
from temp.data.gen import parallel_generate_problem, parallel_generate_solutions
from temp.data.problem import setcover, lot_sizing
from temp.data.info import ModelInfo
from temp.data.dataset import ModelGraphDataset
from temp.data.aug import parallel_augment_info, augment_info


from temp.solver.utils import solve_inst, with_lic


In [4]:
PRE_TRAIN_DIR = "temp/pre_train_tiny"
TRAIN_DIR = "temp/train_tiny"
VALID_DIR = "temp/valid_tiny"

In [5]:
import json
with open("gb.lic") as f:
    lic = json.load(f)

# parallel_generate_problem(lot_sizing, PRE_TRAIN_DIR, 100, 10)
# parallel_generate_solutions(PRE_TRAIN_DIR, 5, lic=lic)

# parallel_generate_problem(lot_sizing, TRAIN_DIR, 100, 10)
# parallel_generate_solutions(TRAIN_DIR, 5, lic=lic)

# parallel_generate_problem(lot_sizing, VALID_DIR, 100, 10)
# parallel_generate_solutions(VALID_DIR, 5, lic=lic)
# assert 1 == 2

In [6]:
# import os
# import shutil

# # Define the source and destination folders
# src_folder = "temp/valid_ls"
# dest_folder = "temp/valid_ls_backup"

# # Create the destination folder if it doesn't exist
# os.makedirs(dest_folder, exist_ok=True)

# # List all files in the source folder
# files = os.listdir(src_folder)

# # First, process all .lp files: if a corresponding .npz file exists, move both to "done"
# for file_name in os.listdir(src_folder):
#     if file_name.endswith(".lp"):
#         lp_path = os.path.join(src_folder, file_name)
#         base_name = os.path.splitext(file_name)[0]
#         npz_file = base_name + ".npz"
#         npz_path = os.path.join(src_folder, npz_file)
        
#         # If the corresponding .npz exists, move both files
#         if os.path.exists(npz_path):
#             shutil.move(lp_path, os.path.join(dest_folder, file_name))
#             shutil.move(npz_path, os.path.join(dest_folder, npz_file))
#             print(f"Moved pair: {file_name} and {npz_file}")

# # Next, remove any .npz files that are left without a corresponding .lp file.
# for file_name in os.listdir(src_folder):
#     if file_name.endswith(".npz"):
#         base_name = os.path.splitext(file_name)[0]
#         lp_file = base_name + ".lp"
#         lp_path = os.path.join(src_folder, lp_file)
#         npz_path = os.path.join(src_folder, file_name)
        
#         if not os.path.exists(lp_path):
#             os.remove(npz_path)
#             print(f"Removed orphan npz: {file_name}")

In [7]:
# m = lot_sizing()
# model = m

# # model.setParam("OutputFlag", 0)
# model.setParam("PoolSolutions", 10)
# model.setParam("PoolSearchMode", 2)
# model = with_lic(model)
# model.optimize()

# vs = model.getVars()
# obj_val_and_sols = []

# for i in range(model.SolCount):
#     # TODO: setting solution number actually takes long time try to optimize
#     # TODO: add function to round by tolerance
#     model.params.SolutionNumber = i
#     obj_val = model.PoolObjVal
#     s = [obj_val] + [v.Xn for v in vs]
#     obj_val_and_sols.append(s)
# model.dispose()

# info = ModelInfo.from_model(m)
# info.var_info.sols = np.array(obj_val_and_sols)

In [8]:
# from temp.data.dataset import info_to_data
# d = info_to_data(info)
# aug = partial(parallel_augment_info, n=100)

In [9]:
# import os
# import pickle
# from temp.data.aug import augment_info, get_constraint_violations, get_lhs_matrix

# for p in os.listdir(os.path.join(PRE_TRAIN_DIR, "processed")):
#     if not p.endswith("pkl"):
#         continue
#     with open(os.path.join(PRE_TRAIN_DIR, "processed", p), "rb") as f:
#         info = pickle.load(f)

#     lhs = get_lhs_matrix(info.var_info.n, info.con_info)
#     rhs = np.array(info.con_info.rhs)
#     ops = np.array(info.con_info.types)
#     violations = get_constraint_violations(lhs, info.var_info.sols[0, 1:], rhs, ops)
#     assert not (np.abs(violations) >= 1e-5).any(), violations

    
#     # for _ in range(100):
#     #     aug_info = info.copy()
#     #     augment_info(aug_info)
#     #     lhs = get_lhs_matrix(aug_info.var_info.n, aug_info.con_info)
#     #     rhs = np.array(aug_info.con_info.rhs)
#     #     ops = np.array(aug_info.con_info.types)
#     #     violations = get_constraint_violations(lhs, aug_info.var_info.sols[0, 1:], rhs, ops)
#     #     assert not (np.abs(violations) >= 1e-5).any(), violations

In [10]:
import gurobipy as gp

from temp.data.info import ModelInfo
from temp.data.problem import lot_sizing, setcover
from temp.solver.utils import with_lic

B_TYP = gp.GRB.BINARY
I_TYP = gp.GRB.INTEGER
C_TYP = gp.GRB.CONTINUOUS


def model_with_one_var_each_type():
    m = gp.Model()
    b = m.addVar(vtype=B_TYP, name="b")
    i = m.addVar(vtype=I_TYP, name="I")
    c = m.addVar(vtype=C_TYP, name="C")
    m.addConstr(b + i + c <= 5)
    m.setObjective(b + i + c, gp.GRB.MAXIMIZE)
    m.update()
    return m


# def model_with_two_var_each_type():
#     m = gp.Model()
#     b1 = m.addVar(vtype=B_TYP, name="b1")
#     b2 = m.addVar(vtype=B_TYP, name="b2")

#     i1 = m.addVar(vtype=I_TYP, name="i1")
#     i2 = m.addVar(vtype=I_TYP, name="i2")

#     c1 = m.addVar(vtype=C_TYP, name="c1")
#     c2 = m.addVar(vtype=C_TYP, name="c2")
#     m.addConstr(b1 + i1 + c1 <= 5)
#     m.addConstr(b2 + i2 + c2 <= 5)
#     m.addConstr(b1 + i1 + c1 + b2 + i2 + c2 >= 5)
#     m.setObjective(b1 + i1 + c1, gp.GRB.MAXIMIZE)
#     m.update()
#     return m


# m = model_with_one_var_each_type()
# # m = model_with_two_var_each_type()
# info = ModelInfo.from_model(m)

# m = with_lic(m)
# m.optimize()
# vals = [x.x for x in m.getVars()]
# sols = [[m.ObjVal] + vals]
# m.dispose()

# info.var_info.sols = np.array(sols)


In [11]:
from temp.data.graph import get_bipartite_graph, add_label
from temp.data.dataset import create_data_object

In [12]:
pre_train_d = ModelGraphDataset(PRE_TRAIN_DIR, augment=augment_info, force_reload=True)
train_d = ModelGraphDataset(PRE_TRAIN_DIR, augment=augment_info)
valid_d = ModelGraphDataset(PRE_TRAIN_DIR)

# for i in range(len(pre_train_d)):
#     print(i)
#     data = pre_train_d[i][1]

data = pre_train_d[0][1]

Processing...
(model, solution) => info: 100%|██████████| 100/100 [00:16<00:00,  6.24it/s]
Done!


In [13]:
var_feature_size = data.var_node_features.size(-1)
con_feature_size = data.con_node_features.size(-1) 

In [14]:
import pandas as pd
CFG = pd.read_excel("temp/trained_models/setcover_model_configs.xlsx", index_col=0).loc[0].to_dict()
CFG["num_epochs"] = 16
CFG["num_layers"] = 8
CFG["hidden"] = 32

In [15]:
from temp.model.utils import get_model
from temp.model.trainer import train

In [16]:
model_name, model, criterion, optimizer, scheduler = get_model(".", var_feature_size, con_feature_size, n_batches=1, **CFG)

In [17]:
scheduler.total_steps = 10000

In [18]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed()
    np.random.seed(worker_seed)
    random.seed(worker_seed)

from torch_geometric.loader import DataLoader
pretrain_loader = DataLoader(pre_train_d, batch_size=2, shuffle=True, num_workers=8, prefetch_factor=2, generator=torch.Generator().manual_seed(0))
train_loader = DataLoader(train_d, batch_size=2, shuffle=True, num_workers=8, prefetch_factor=2, generator=torch.Generator().manual_seed(0))
val_loader = DataLoader(valid_d, batch_size=2, shuffle=True, num_workers=8, generator=torch.Generator().manual_seed(0))

In [19]:
scheduler.total_steps = 1000000

In [20]:
train(model_name, model, criterion, optimizer, scheduler, pretrain_loader, train_loader, val_loader, CFG, False, "./")

>> Training starts on the current device cpu
>> Pretraining for prenorm...


 88%|████████▊ | 44/50 [03:37<00:13,  2.17s/it] 

In [ ]:
assert 1 == 2

AssertionError: 

In [22]:
from temp.data.dataset import info_to_data

In [48]:
m = lot_sizing(128, 16)
to_dispose = []

In [74]:
gb_m = with_lic(m)
gb_m.setParam("TimeLimit", 150)
# gb_m.setParam("NoRelHeurTime", 60)
gb_m.optimize()
to_dispose.append(gb_m)

Set parameter CloudAccessID
Set parameter CloudSecretKey
Set parameter CloudPool to value "831775-C3Dev"
Set parameter CSAppName to value "Josh"
Compute Server job ID: 595213e8-92ff-4843-8d1e-48b70cd9283c
Capacity available on '831775-C3Dev' cloud pool - connecting...
Established HTTPS encrypted connection

Compute Server communication statistics:
  Sent: 0.098 MB in 57 msgs and 4.40s (0.02 MB/s)
  Received: 1.146 MB in 345 msgs and 146.77s (0.01 MB/s)

Set parameter TimeLimit to value 150
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 22.4.0 22E252)
Gurobi Compute Server Worker version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 20.04.6 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8375C CPU @ 2.90GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
TimeLimit  150
CSIdleTimeout  1800

Optimize a model with 6160 rows, 6144 columns and 16256 nonzeros
Model fingerprint: 0x384

In [51]:
info = ModelInfo.from_model(m)
data = info_to_data(info)

In [52]:
with torch.no_grad():
    logits = model(data)

In [53]:
len(logits)

6144

In [54]:
def get_binary_mask(model):
    vs = model.getVars()
    ts = model.getAttr("vType", vs)
    mask = [t == gp.GRB.BINARY for t in ts]
    mask = torch.as_tensor(mask, dtype=bool)
    return mask

def get_binary_indices(model):
    vs = model.getVars()
    ts = model.getAttr("vType", vs)
    idxs = [i for i, t in enumerate(ts) if t == gp.GRB.BINARY]
    idxs = torch.as_tensor(idxs, dtype=int)
    return idxs

In [55]:
binary_mask = get_binary_mask(m)

In [56]:
from temp.solver import upr

In [57]:
probs, preds = upr.get_predictions(logits, binary_mask)

In [58]:
preds

array([1.0371833e-07, 3.1710293e-08, 3.1230822e-08, ..., 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00], dtype=float32)

In [59]:
unc = upr.get_uncertainty(logits)
unc = unc[binary_mask]

In [60]:
threshold = upr.get_threshold(unc)

In [61]:
import json
from tempfile import NamedTemporaryFile

import gurobipy as gp


def fix_var(inst, idxs, vals):
    assert len(idxs) == len(vals)
    bounds = {}
    vs = inst.getVars()
    for idx, val in zip(idxs, vals):
        v = vs[idx]
        bounds[idx] = (v.lb, v.ub)
        v.setAttr("lb", val)
        v.setAttr("ub", val)
    inst.update()
    return bounds


def unfix_var(inst, idxs, bounds):
    assert len(idxs) == len(bounds)
    vs = inst.getVars()
    print(idxs, bounds)
    for i, (lb, ub) in zip(idxs, bounds):
        vs[i].setAttr("lb", lb)
        vs[i].setAttr("ub", ub)


def get_iis_vars(inst):
    try:
        inst.computeIIS()
    except Exception as e:
        print(e)
        if "Cannot compute IIS on a feasible model" in str(e):
            return set()
        raise e

    with NamedTemporaryFile(suffix=".ilp", mode="w+") as f:
        inst.write(f.name)
        f.seek(0)
        return set(f.read().split())


def set_starts(inst, starts):
    vs = inst.getVars()
    for i, s in starts.items():
        vs[i].setAttr("lb", s)


def solve_inst(inst):
    vs = inst.getVars()
    inst.optimize()
    return inst.getAttr("X", vs)


def repair(inst, fixed: set, bounds: dict):
    old_iis_method = getattr(inst, "IISMethod", -1)
    # inst.setParam("IISMethod", 0)

    vs = inst.getVars()
    ns = inst.getAttr("varName", vs)
    name_to_idx = {n: i for i, n in enumerate(ns)}

    freed = set()
    while iis_var_names := get_iis_vars(inst):
        for n in iis_var_names:
            if n not in name_to_idx:
                continue
            
            var_idx = name_to_idx[n]
            if var_idx not in fixed:
                continue

            if var_idx in freed:
                continue
            print(n, var_idx)
            if var_idx not in bounds:
                continue
            
            lb, ub = bounds[var_idx]
            vs[var_idx].lb = lb
            vs[var_idx].ub = ub
            freed.add(var_idx)

    inst.setParam("IISMethod", old_iis_method)
    return freed


def set_hints(inst, hints):
    vs = inst.getVars()
    for i, (val, pri) in hints:
        if 0 <= pri <= 1:
            pri = int(pri * 100)
        vs[i].setAttr("VarHintVal", val)
        vs[i].setAttr("VarHintPri", pri)


def with_lic(m, path="gb.lic"):
    with open(path) as f:
        env = gp.Env(params=json.load(f))
    return m.copy(env=env)


In [62]:
print(m.getVarByName("y[5,15]").ub)
print(m.getVarByName("y[5,15]").lb)

1.0
0.0


In [71]:
import numpy as np
import torch
import torch.nn.functional as F

# from temp.solver.utils import fix_var, repair, set_starts, unfix_var, solve_inst

EVIDENCE_FUNCS = {
    "softplus": (lambda y: F.softplus(y)),
    "relu": (lambda y: F.relu(y)),
    "exp": (lambda y: torch.exp(torch.clamp(y, -10, 10))),
}


def get_predictions(logits, binary_mask):
    probs = torch.softmax(logits, axis=1)
    preds = probs[:, 1]
    probs = _to_numpy(probs)
    preds = _to_numpy(preds).squeeze()
    preds[binary_mask] = preds[binary_mask].round()
    return probs, preds


def get_uncertainty(logits, evidence_func_name: str = "softplus"):
    evidence = EVIDENCE_FUNCS[evidence_func_name](logits)
    alpha = evidence + 1
    uncertainty = logits.shape[1] / torch.sum(alpha, dim=1, keepdim=True)
    return uncertainty


def get_threshold(uncertainty: torch.Tensor, r_min: float = 0.4, r_max: float = 0.55):
    q = (r_min + r_max) / 2
    threshold = torch.quantile(uncertainty, q)
    r = (uncertainty <= threshold).float().mean()

    if r > r_max:
        threshold = torch.quantile(uncertainty, r_max)
        return threshold

    if r < r_min:
        threshold = torch.quantile(uncertainty, r_min)
        return threshold

    return threshold


def get_confident_idx(indices, uncertainty, threshold):
    confident_mask = uncertainty <= threshold
    confident_idx = indices[confident_mask]
    return confident_idx.sort()[0]

import os

def solve(inst, prediction, uncertainty, indices, max_iter):
    threshold = get_threshold(uncertainty)
    conf_idxs = get_confident_idx(indices, uncertainty, threshold)
    conf_vals = prediction[conf_idxs]

    conf_idxs = conf_idxs.tolist()
    bounds = fix_var(inst, conf_idxs, conf_vals)
    
    os.bnds = bounds
    os.idxs = conf_idxs
    os.vals = conf_vals

    print(len(bounds), len(prediction))

    min_q = sum(uncertainty <= threshold) / len(uncertainty)
    max_q = 1.0
    dq = (max_q - min_q) / (max_iter - 1)

    fixed = set(conf_idxs)
    freed = set(repair(inst, fixed, bounds))
    for i in range(1, max_iter):
        sol = solve_inst(inst)
        q = max_q - dq * i
        threshold = np.quantile(uncertainty, q)
        conf_idxs = get_confident_idx(indices, uncertainty, threshold)
        to_unfix = list(fixed - set(conf_idxs.tolist()))
        to_unfix = [i for i in to_unfix if i not in freed]
        print(len(to_unfix))
        unfix_var(inst, to_unfix, [bounds[i] for i in to_unfix])
        starts = {i: sol[i] for i in to_unfix}
        starts.update({i: sol[i] for i in freed})
        set_starts(inst, starts)
    unfix_var(inst, bounds.keys(), bounds.values())
    sol = solve_inst(inst)
    return sol


def _to_numpy(tensor_obj):
    return tensor_obj.cpu().detach().numpy()


In [72]:
for d in to_dispose:
    d.dispose()


Compute Server communication statistics:
  Sent: 0.110 MB in 122 msgs and 9.32s (0.01 MB/s)
  Received: 1.212 MB in 548 msgs and 201.05s (0.01 MB/s)



In [73]:
indices = get_binary_indices(m)
unc = unc.squeeze()
to_solve = with_lic(m)
to_dispose.append(to_solve)
to_solve.setParam("TimeLimit", 10)
solve(to_solve, preds, unc, indices, 5)

Set parameter CloudAccessID
Set parameter CloudSecretKey
Set parameter CloudPool to value "831775-C3Dev"
Set parameter CSAppName to value "Josh"
Compute Server job ID: c9d5fa46-c9b5-4b8e-9e9c-676eb9ff4c53
Capacity available on '831775-C3Dev' cloud pool - connecting...
Established HTTPS encrypted connection
Set parameter TimeLimit to value 10
973 6144
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 22.4.0 22E252)
Gurobi Compute Server Worker version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 20.04.6 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8375C CPU @ 2.90GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
TimeLimit  10
CSIdleTimeout  1800

IIS computation: initial model status unknown, solving to determine model status

Computing Irreducible Inconsistent Subsystem (IIS)...

           Constraints          |            Bounds           |  Runtime
      Min       Max   

[5086.771200000001,
 1695.5904000000003,
 1695.5904000000003,
 847.7952000000001,
 423.89760000000007,
 423.89760000000007,
 167.17688340479998,
 83.58844170239999,
 29.023764479999986,
 14.511882239999998,
 6.046617599999999,
 5.038848,
 2.099519999999998,
 0.5831999999999999,
 0.486,
 0.20249999999999999,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 44.771916595200054,
 22.385958297600027,
 23.963435520000022,
 11.981717760000006,
 7.2001824000000125,
 3.5734560000000326,
 2.2066320000000186,
 0.7905599999999997,
 0.4836000000000037,
 0.27449999999999997,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 4.634496,
 2.317248,
 0.6164400000000264,
 0.5364,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.21759999999984436,
 0.0,
 0.276000000000002,
 0